### Pull Tract-Level and County-Level ACS 5-year data using Census library and API

In [1]:
data_path = 'data'

Mounted at /drive


In [2]:

#Used this notebook as inspiration: 
#https://github.com/BuzzFeedNews/2020-02-gentrification/blob/master/notebooks/01-download-census-data.ipynb. 

# shapefiles available by state: 
# https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2021&layergroup=Places

In [3]:
#installations
!pip install census
!pip install us

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 5.1 MB/s 
  Created wheel for us: filename=us-2.0.2-py3-none-any.whl size=11945 sha256=05d2af4c1ab7887baeaf57b493364bb4d92c9b0c6b185699c58f9e980e405ec2
  Stored in directory: /root/.cache/pip/wheels/ca/6b/11/cda9ea2438f721330a35c9a2c8e34b4aedcd34c89af48a4d00
  Created wheel for jellyfish: filename=jellyfish-0.6.1-cp37-cp37m-linux_x86_64.whl size=72179 sha256=309b217ed731b85afc41a124f25dd39eb47357ec3664e80c4e3d300f762a0a74
  Stored in directory: /root/.cache/pip/wheels/a1/99/51/7de469e37cd1b3c763c24394e1ebf1baa2d79e094bf346cf80
Successfully built us jellyfish


In [4]:
# Dependencies

import pandas as pd
import numpy as np
import requests
from census import Census
from us import states

### Pull Census 5-year data and select variables

In [5]:
c = Census('307a9c458f89f32c9cf817fb1ae4f426ccc84e2c')
#This is the key to use the API. It was obtained at https://api.census.gov/data/key_signup.html


In [6]:
#List of variables available at https://api.census.gov/data/2019/acs/acs1/variables.html 

rename_dict = {
     'NAME': 'name',
     'B01001_001E': "population",
     #'B02001_001E': 'population_race',
     'B02001_002E': "white", # for all races chose people who identified only as that race B01001A_001E
     'B02001_003E': "black",
     'B02001_004E': "native_american",
     'B02001_005E': "asian",
     'B02001_006E': "hawaiian",
     'B02001_007E': "some_other_race_alone",
     'B02001_008E': "two_more_races",
     'B03002_012E': "hispanic_or_latino", 
     'B01001_004E': 'male_5_9',
     'B01001_005E': 'male_10_14',
     'B01001_006E': 'male_15_17',
     'B01001_028E': 'female_5_9',
     'B01001_029E': 'female_10_14',
     'B01001_030E': 'female_15_17',
     'B01001_011E': "male_25_29",
     'B01001_012E': "male_30_34",
     'B01001_013E': 'male_35_39',
     'B01001_014E': 'male_40_44',
     'B01001_015E': 'male_45_49',
     'B01001_016E': "male_50_54",
     'B01001_035E': "female_25_29",
     'B01001_036E': "female_30_34",
     'B01001_037E': 'female_35_39',
     'B01001_038E': 'female_40_44',
     'B01001_039E': 'female_45_49',
     'B01001_040E': "female_50_54",
     'B23001_003E': 'male_16_19',
     'B23001_007E': 'male_16_19_labor_force_employed',
     'B23001_010E': 'male_20_21',
     'B23001_014E': 'male_20_21_labor_force_employed',
     'B23001_017E': 'male_22_24', 
     'B23001_021E': 'male_22_24_labor_force_employed',
     'B23001_089E': 'female_16_19',
     'B23001_093E': 'female_16_19_labor_force_employed',
     'B23001_096E': 'female_20_21',
     'B23001_100E': 'female_20_21_labor_force_employed',
     'B23001_103E': 'female_22_24', 
     'B23001_107E': 'female_22_24_labor_force_employed',
     'B23001_138E': 'female_55_59',
     'B23001_142E': 'female_55_59_labor_force_employed',
     'B23001_145E': 'female_60_61', 
     'B23001_149E': 'female_60_61_labor_force_employed',
     'B23001_152E': 'female_62_64', 
     'B23001_156E':  'female_62_64_labor_force_employed',
     'B23001_052E': 'male_55_59',
     'B23001_056E': 'male_55_59_labor_force_employed',
     'B23001_059E': 'male_60_61', 
     'B23001_063E': 'male_60_61_labor_force_employed',
     'B23001_066E': 'male_62_64', 
     'B23001_070E': 'male_62_64_labor_force_employed',
     'B23001_028E':'male_25_29_labor_force_employed',
     'B23001_035E':'male_30_34_labor_force_employed',
     'B23001_042E':'male_35_44_labor_force_employed',
     'B23001_042E':'male_35_44_labor_force_employed',
     'B23001_049E':'male_45_54_labor_force_employed',
     'B23001_114E':'female_25_29_labor_force_employed', #Total:!!Female:!!25 to 29 years:!!In labor force:!!Civilian:!!Employed
     'B23001_121E':'female_30_34_labor_force_employed',
     'B23001_128E':'female_35_44_labor_force_employed',
     'B23001_135E':'female_45_54_labor_force_employed',
     'C23002A_003E': 'white_male_16_64',
     'C23002A_007E': 'white_male_16_64_employed',
     'C23002A_016E': 'white_female_16_64',
     'C23002A_020E': 'white_female_16_64_employed',
     'C23002B_003E': 'black_male_16_64',
     'C23002B_007E': 'black_male_16_64_employed',
     'C23002B_016E': 'black_female_16_64',
     'C23002B_020E': 'black_female_16_64_employed',     
     'C23002C_003E': 'american_indian_male_16_64',
     'C23002C_007E': 'american_indian_male_16_64_employed',
     'C23002C_016E': 'american_indian_female_16_64',
     'C23002C_020E': 'american_indian_female_16_64_employed',
     'C23002D_003E': 'asian_male_16_64',
     'C23002D_007E': 'asian_male_16_64_employed',
     'C23002D_016E': 'asian_female_16_64',
     'C23002D_020E': 'asian_female_16_64_employed',
     'C23002F_003E': 'some_other_race_alone_male_16_64',
     'C23002F_007E': 'some_other_race_alone_male_16_64_employed',
     'C23002F_016E': 'some_other_race_alone_female_16_64',
     'C23002F_020E': 'some_other_race_alone_female_16_64_employed',
     'C23002G_003E': 'two_or_more_race_male_16_64',
     'C23002G_007E': 'two_or_more_race_male_16_64_employed',
     'C23002G_016E': 'two_or_more_race_female_16_64',
     'C23002G_020E': 'two_or_more_race_female_16_64_employed',
     'C23002I_003E': 'hispanic_or_latino_male_16_64',
     'C23002I_007E': 'hispanic_or_latino_male_16_64_employed',
     'C23002I_016E': 'hispanic_or_latino_female_16_64',
     'C23002I_020E': 'hispanic_or_latino_female_16_64_employed',
     'B19113_001E': "median_family_income", # MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)
     'B19080_001E': "income_20_percentile", ### 20th percentile total household income
     'B19080_004E': "income_80_percentile", ### 80th percentile total household income
     'B19113A_001E':"median_family_income_white", #(WHITE ALONE HOUSEHOLDER)
     'B19113B_001E':"median_family_income_black", #BLACK OR AFRICAN AMERICAN ALONE HOUSEHOLDER
     'B19113C_001E':'median_family_income_indigenous',#AMERICAN INDIAN AND ALASKA NATIVE ALONE HOUSEHOLDER
     'B19113D_001E':'median_family_income_asian', #ASIAN ALONE HOUSEHOLDER
     'B19113H_001E':'median_family_income_white_not_hisp', #WHITE ALONE, NOT HISPANIC OR LATINO HOUSEHOLDER
     'B19113I_001E':"median_family_income_hispanic",#HISPANIC OR LATINO HOUSEHOLDER
     #'B25111_001E': "median_rent",
     'B01001_027E': 'female_under5',
     'B01001_003E':'male_under5',
     'B01001A_003E':'white_m_under5', 
     'B01001A_018E': 'white_f_under5',
     'B01001B_003E': 'black_m_under5',
     'B01001B_018E':'black_f_under5',
     'B01001C_003E':'indigenous_m_under5', #SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)
     'B01001C_018E':'indigenous_f_under5', #SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)
     'B01001D_003E':'asian_m_under5',
     'B01001D_018E':'asian_f_under5',
     'B01001E_003E':'hawaiian_m_under5', 
     'B01001E_018E':'hawaiian_f_under5', #SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)
     'B01001F_003E':'some_other_race_m_under5',
     'B01001F_018E':'some_other_race_f_under5',
     'B01001G_003E':'two_or_more_race_m_under5',
     'B01001G_018E':'two_or_more_race_f_under5',
     'B01001I_003E':'hispanic_m_under5',
     'B01001I_018E':'hispanic_f_under5',
     'B09001_004E': 'total_3_4', # number of 3 and 4 year olds
     'B14001_003E': 'preschool_enrollment', #Enrolled in nursery school, preschool 3+ - this could include 5 year olds
     'B14007A_003E': 'preschool_enrollment_white',
     'B14007B_003E': 'preschool_enrollment_black',
     'B14007I_003E':'preschool_enrollment_hispanic',
     'B14007C_003E': 'preschool_enrollment_indigenous',
     'B14007D_003E': 'preschool_enrollment_asian',
     'B14003_050E': "fem_not_enrolled_3_4",
     'B14003_022E': "mal_not_enrolled_3_4",
     #'B25071_001E': "rent_burden",  ###Rent Burden (MEDIAN GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME)
     #'B25077_001E': "median_home_value",
      # https://www.census.gov/topics/income-poverty/poverty/guidance/poverty-measures.html
     'B17001_001E': 'total_poverty', # number of people for whom poverty status exists
     'B17001_002E':'all_in_poverty', # number of people in poverty
     'B17001_031E': 'all_not_poverty', # number of people not in poverty Income in the past 12 months at or above poverty level
     'B05010_001E': "ratio_of_income_to_poverty",# BY NATIVITY OF CHILDREN UNDER 18 YEARS IN FAMILIES AND SUBFAMILIES BY LIVING ARRANGEMENTS AND NATIVITY OF PARENTS
     'B05010_002E': 'total_poverty_ratio_under1_children', #this is for children
     'B05010_010E': 'total_poverty_ratio_1_to_2_children',
     'B05010_018E': 'total_poverty_ratio_2_or_more_children',
     'B17026_001E': 'ratio_income_to_poverty_families', # Estimate!!Total:	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_002E': "poverty_ratio_under.5", # Under .50	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_003E':'poverty_ratio_.5_.74',  # .50 to .74	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_004E': 'poverty_ratio_.75_.99', # .75 to .99	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_005E':'poverty_ratio_1_1.24', # 1.00 to 1.24	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_006E': 'poverty_ratio_1.25_1.49', # 1.25 to 1.49	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_007E': 'poverty_ratio_1.5_1.74', # 1.50 to 1.74	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_008E': 'poverty_ratio_1.75_1.84',
     'B17026_009E': 'poverty_ratio_1.85_1.99', # 1.85 to 1.99	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS  
     #'B08202_001E': 'numer_households', # I think???
     'B14002_005E': 'male_public_preschool',
     'B14002_008E':'male_public_kinder',
     'B14002_011E': 'male_public_1_4',
     'B14002_014E': 'male_public_5_8',
     'B14002_017E': 'male_public_9_12',
     'B14002_029E': 'female_public_preschool',
     'B14002_032E':'female_public_kinder',
     'B14002_035E': 'female_public_1_4',
     'B14002_038E': 'female_public_5_8',
     'B14002_041E': 'female_public_9_12',
     #'B29001_001E': 'total_citizen_voting_age' #CITIZEN, VOTING-AGE POPULATION BY AGE
     }

categories = list(rename_dict.keys())



### Voting Eligible population

In [7]:
# https://pypi.org/project/census/ list of geography levels available
#This function pulls county level data for acs 5-year


acs_5yr = c.acs5.state_county(
        ['B29001_001E'], #voting age population
        Census.ALL,
        #state_code,
        Census.ALL,
        year = 2020
    )
voting_population = pd.DataFrame(acs_5yr) # this is county-level data for this variable for all counties in the US
voting_population = voting_population.rename(columns={'B29001_001E':'voting_population'})
voting_population.head()

,voting_population,state,county
0,41959.0,01,001
1,167986.0,01,003
2,19468.0,01,005
3,17583.0,01,007
4,42978.0,01,009


In [8]:
core_cities_dict = {'14140':'Central Falls','54640':'Pawtucket','59000':'Providence','80780':'Woonsocket'} # Central falls, pawtucket, providence, woonsocket

acs_5yr = c.acs5.state_place(
        ['B29001_001E'], #voting age population
        '44', #state_code,
        Census.ALL,
        year = 2020
    )
voting_population_ri_places = pd.DataFrame(acs_5yr) # this is county-level data for this variable for all counties in the US
voting_population_ri_places = voting_population_ri_places.rename(columns={'B29001_001E':'voting_population'})
voting_population_ri_places["city"] = voting_population_ri_places["place"].map(core_cities_dict)

voting_population_ri_places = voting_population_ri_places[voting_population_ri_places['place'].isin(list(core_cities_dict.keys()))]
voting_population_ri_places.head()

,voting_population,state,place,city
3,9825.0,44,14140,Central Falls
24,50599.0,44,54640,Pawtucket
25,116420.0,44,59000,Providence
34,30024.0,44,80780,Woonsocket


### Variables dictionaries

In [9]:
# # See reference map for an overview of all census tracts in RI https://www2.census.gov/geo/maps/dc10map/tract/st44_ri/c44007_providence/DC10CT_C44007_002.pdf
# #info on RI tracts/Counties https://en.wikipedia.org/wiki/List_of_counties_in_Rhode_Island
# d={}
# d['county_name']=["Bristol County","Kent County","Newport County","Providence County","Washington County"]
# d['full_county_geocode']=['36005','36047','36061','36081','36085'] # see second link
# d['state_code']=['44','44','44','44','44']
# d['county_code']=['001','003','005','007','009']
# d["city"]=['Bristol','Kent','Newport','Providence','Washington']
# d["metro_area_name"]=['RI','RI','RI','RI','RI']

# metro_area_counties=pd.DataFrame(d)
# #metro_area_counties


In [10]:
rename_dict_tracts = {
     'NAME': 'name',
     'B01001_001E': "population",
     'B17001_001E': 'total_poverty', # number of people for whome poverty status exists
     'B17001_002E':'all_in_poverty', # number of people in poverty
     'B17001_031E': 'all_not_poverty', # number of people not in poverty Income in the past 12 months at or above poverty level
     'B05010_001E': "ratio_of_income_to_poverty",# BY NATIVITY OF CHILDREN UNDER 18 YEARS IN FAMILIES AND SUBFAMILIES BY LIVING ARRANGEMENTS AND NATIVITY OF PARENTS
     'B05010_002E': 'total_poverty_ratio_under1_children', #this is for children
     'B05010_010E': 'total_poverty_ratio_1_to_2_children',
     'B05010_018E': 'total_poverty_ratio_2_or_more_children',
     'B17026_001E': 'ratio_income_to_poverty_families', # Estimate!!Total:	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_002E': "poverty_ratio_under.5", # Under .50	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_003E':'poverty_ratio_.5_.74',  # .50 to .74	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_004E': 'poverty_ratio_.75_.99', # .75 to .99	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_005E':'poverty_ratio_1_1.24', # 1.00 to 1.24	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_006E': 'poverty_ratio_1.25_1.49', # 1.25 to 1.49	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_007E': 'poverty_ratio_1.5_1.74', # 1.50 to 1.74	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS
     'B17026_008E': 'poverty_ratio_1.75_1.84',
     'B17026_009E': 'poverty_ratio_1.85_1.99', # 1.85 to 1.99	RATIO OF INCOME TO POVERTY LEVEL OF FAMILIES IN THE PAST 12 MONTHS  
     'B02001_002E': "white", # for all races chose people who identified only as that race B01001A_001E
     'B02001_003E': "black",
     'B02001_004E': "native_american",
     'B02001_005E': "asian",
     'B02001_006E': "hawaiian",
     'B02001_007E': "some_other_race_alone",
     'B02001_008E': "two_more_races",
     'B03002_012E': "hispanic_or_latino", 
}

categories_tracts = list(rename_dict_tracts.keys())


In [11]:
state_fips = ['01', '02', '04', '05', '06', '08', '09', '10', '12', '13', '15', '16', '17',
              '18', '19', '20', '21', '22','23','24', '25', '26','27','28','29','30','31',
              '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45',
              '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '60', '66', '69',
              '72', '78']


In [12]:
timeperiod = 2019
us_tract_df = pd.DataFrame()

for fips in state_fips:
    acs_5yr = c.acs5.state_county_tract(
        categories_tracts,
        fips, 
        Census.ALL, 
        Census.ALL,
        year = timeperiod
    )
    df = pd.DataFrame(acs_5yr)

   

   
    df = df.rename(columns=rename_dict_tracts)
    df['year'] = timeperiod

    # Concatenate
    us_tract_df = pd.concat([us_tract_df,df], ignore_index=True)


us_tract_df.head()



,name,population,total_poverty,all_in_poverty,all_not_poverty,ratio_of_income_to_poverty,total_poverty_ratio_under1_children,total_poverty_ratio_1_to_2_children,total_poverty_ratio_2_or_more_children,ratio_income_to_poverty_families,...,native_american,asian,hawaiian,some_other_race_alone,two_more_races,hispanic_or_latino,state,county,tract,year
0,"Census Tract 11, Jefferson County, Alabama",4781.0,4655.0,700.0,3955.0,1069.0,352.0,248.0,469.0,1213.0,...,58.0,0.0,0.0,0.0,0.0,17.0,01,073,001100,2019
1,"Census Tract 14, Jefferson County, Alabama",1946.0,1946.0,548.0,1398.0,339.0,146.0,19.0,174.0,406.0,...,0.0,0.0,0.0,151.0,7.0,133.0,01,073,001400,2019
2,"Census Tract 20, Jefferson County, Alabama",4080.0,4007.0,2057.0,1950.0,881.0,634.0,159.0,88.0,714.0,...,0.0,0.0,0.0,82.0,38.0,306.0,01,073,002000,2019
3,"Census Tract 38.02, Jefferson County, Alabama",5291.0,5291.0,1040.0,4251.0,1255.0,439.0,490.0,326.0,1301.0,...,0.0,0.0,0.0,0.0,45.0,0.0,01,073,003802,2019
4,"Census Tract 40, Jefferson County, Alabama",2533.0,2533.0,1028.0,1505.0,236.0,111.0,73.0,52.0,513.0,...,22.0,0.0,0.0,0.0,114.0,173.0,01,073,004000,2019


### City level data (places) 
Core Cities

In [13]:
core_cities_codes = ['14140','54640','59000','80780'] # Central falls, pawtucket, providence, woonsocket

all_df_place = pd.DataFrame()

for timeperiod in range(2019, 2020):
    acs_5yr = c.acs5.state_place(
        categories,
        '44', #RI FIPS Code
        Census.ALL,
        year = timeperiod
    )

    df = pd.DataFrame(acs_5yr)

    df = df.rename(columns=rename_dict)
    df['year'] = timeperiod

    # Concatenate
    all_df_place = pd.concat([all_df_place,df], ignore_index=True)



all_df_place = all_df_place[all_df_place['place'].isin(core_cities_codes)]

In [14]:
all_df_place[['GEO_ID','name','place']].head(4)



,GEO_ID,name,place
3,1600000US4414140,"Central Falls city, Rhode Island",14140
24,1600000US4454640,"Pawtucket city, Rhode Island",54640
25,1600000US4459000,"Providence city, Rhode Island",59000
34,1600000US4480780,"Woonsocket city, Rhode Island",80780


In [15]:

core_cities = ['Providence city, Rhode Island', 'Central Falls city, Rhode Island', 'Woonsocket city, Rhode Island','Pawtucket city, Rhode Island']



all_df_place.columns

Index(['name', 'population', 'white', 'black', 'native_american', 'asian',
       'hawaiian', 'some_other_race_alone', 'two_more_races',
       'hispanic_or_latino',
       ...
       'male_public_kinder', 'male_public_1_4', 'male_public_5_8',
       'male_public_9_12', 'female_public_preschool', 'female_public_kinder',
       'female_public_1_4', 'female_public_5_8', 'female_public_9_12', 'year'],
      dtype='object', length=154)

# US County-Level Data

In [16]:
all_df_county = pd.DataFrame()

for timeperiod in range(2019, 2020):
    acs_5yr = c.acs5.state_county(
        categories,
        Census.ALL,
        Census.ALL,
        year = timeperiod
    )

    df_county = pd.DataFrame(acs_5yr)

    df_county = df_county.rename(columns=rename_dict)
    df_county['year'] = timeperiod

# Cleaning Data

In [17]:
all_df_place = all_df_place.replace(-666666666,np.nan)
all_df_place.describe()

,population,white,black,native_american,asian,hawaiian,some_other_race_alone,two_more_races,hispanic_or_latino,male_5_9,...,male_public_kinder,male_public_1_4,male_public_5_8,male_public_9_12,female_public_preschool,female_public_kinder,female_public_1_4,female_public_5_8,female_public_9_12,year
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.0
mean,78092.500000,45781.500000,12319.500000,708.250000,3762.000000,103.250000,11402.000000,4016.000000,29156.250000,2365.250000,...,491.750000,1762.000000,1708.000000,1829.750000,240.000000,387.000000,1689.250000,1473.750000,1528.500000,2019.0
std,70932.356972,38163.900015,12739.103723,721.155265,4780.400471,111.017641,12202.629252,3122.173068,32673.768963,1996.942726,...,340.729976,1467.742711,1358.886063,1791.649775,292.939129,358.302479,1463.804489,1253.468621,1399.433814,0.0
min,19429.000000,9424.000000,2460.000000,196.000000,126.000000,0.000000,1958.000000,1375.000000,7818.000000,755.000000,...,187.000000,621.000000,600.000000,509.000000,44.000000,174.000000,453.000000,521.000000,669.000000,2019.0
25%,36059.500000,25305.250000,3606.000000,357.250000,1320.750000,48.000000,4808.750000,2017.000000,11631.750000,1130.000000,...,295.750000,831.000000,828.750000,848.000000,95.000000,203.250000,851.250000,658.250000,761.250000,2019.0
50%,56723.500000,37433.000000,8303.000000,430.000000,2067.500000,76.500000,7206.000000,3152.500000,15525.000000,1745.500000,...,404.500000,1282.500000,1304.500000,1173.500000,120.000000,225.500000,1261.000000,1054.500000,915.000000,2019.0
75%,98756.500000,57909.250000,17016.500000,781.000000,4508.750000,131.750000,13799.250000,5151.500000,33049.500000,2980.750000,...,600.500000,2213.500000,2183.750000,2155.250000,265.000000,409.250000,2099.000000,1870.000000,1682.250000,2019.0
max,179494.000000,98836.000000,30212.000000,1777.000000,10787.000000,260.000000,29238.000000,8384.000000,77757.000000,5215.000000,...,971.000000,3862.000000,3623.000000,4463.000000,676.000000,923.000000,3782.000000,3265.000000,3615.000000,2019.0


In [18]:
df_county = df_county.replace(-666666666,np.nan)
us_tract_df = us_tract_df.replace(-666666666,np.nan)

In [19]:
for column in list(all_df_place.columns):
  print(column)

name
population
white
black
native_american
asian
hawaiian
some_other_race_alone
two_more_races
hispanic_or_latino
male_5_9
male_10_14
male_15_17
female_5_9
female_10_14
female_15_17
male_25_29
male_30_34
male_35_39
male_40_44
male_45_49
male_50_54
female_25_29
female_30_34
female_35_39
female_40_44
female_45_49
female_50_54
male_16_19
male_16_19_labor_force_employed
male_20_21
male_20_21_labor_force_employed
male_22_24
male_22_24_labor_force_employed
female_16_19
female_16_19_labor_force_employed
female_20_21
female_20_21_labor_force_employed
female_22_24
female_22_24_labor_force_employed
female_55_59
female_55_59_labor_force_employed
female_60_61
female_60_61_labor_force_employed
female_62_64
female_62_64_labor_force_employed
male_55_59
male_55_59_labor_force_employed
male_60_61
GEO_ID
state
place
male_60_61_labor_force_employed
male_62_64
male_62_64_labor_force_employed
male_25_29_labor_force_employed
male_30_34_labor_force_employed
male_35_44_labor_force_employed
male_45_54_labor_f

### Under 18

In [20]:
def under_18(df):
  columns = ['female_under5', 'female_5_9', 'female_10_14', 'female_15_17', 'male_under5', 'male_5_9', 'male_10_14', 'male_15_17']

  col_to_drop = ['female_5_9', 'female_10_14', 'female_15_17', 'male_5_9', 'male_10_14', 'male_15_17']
  
  # add all columns
  df['under_18'] = df[columns].sum(axis=1)

  df.drop(columns = col_to_drop, inplace = True)

  return df

all_df_place = under_18(all_df_place)
df_county = under_18(df_county)

### Diversity Metric
See [here](https://archives.huduser.gov/healthycommunities/sites/default/files/public/Racial%20Diversity%20using%20Shannon-Wiener%20Index.pdf) for calculation instructions - it can be calculated at both the city and neighborhood level. 

In [21]:
import math

def diversity_metric(df):
  races = ['white', 'black', 'native_american', 'asian', 'hawaiian','some_other_race_alone','two_more_races']
  ethnicity = ['hispanic_or_latino']

  # #add all of the people who identify as different races
  # df['total_races'] = df[races].sum(axis=1)
  
  for race in races:
    df[race] = df[race] / df['population']

  df['hispanic_or_latino'] = df['hispanic_or_latino'] /df['population']
  # races = ['white', 'black', 'native_american', 'asian',
  #     # 'some_other_race_alone', 'two_more_races',
  #      'hispanic_or_latino']

  # list_ln = []
  # list_5c = []
  # # step 5b and 5c of tutorial - calculate log natural of each percent
  # for race in races:
  #   df[race+'_ln'] = df[race].apply(lambda x: math.log(x) if x != 0 else 0)
  #   df[race+'_5c'] = df[race]*df[race+'_ln']
  #   list_5c.append(race+'_5c')
  #   list_ln.append(race+'_ln')

  # df['diversity_index'] = -df[list_5c].sum(axis=1) # I think this should be normalized


  # df.drop(axis=1, columns=list_ln, inplace=True)
  # df.drop(axis=1, columns=list_5c, inplace=True)

  return df

all_df_place = diversity_metric(all_df_place)

df_county = diversity_metric(df_county)


#us_tract_df = diversity_metric(us_tract_df)


In [22]:
all_df_place[['name','population', 'year', 'white', 'black', 'native_american', 'asian', 'hawaiian','some_other_race_alone','two_more_races','hispanic_or_latino']]

  

,name,population,year,white,black,native_american,asian,hawaiian,some_other_race_alone,two_more_races,hispanic_or_latino
3,"Central Falls city, Rhode Island",19429.0,2019,0.485048,0.126615,0.010088,0.006485,0.004581,0.296413,0.070770,0.664110
24,"Pawtucket city, Rhode Island",71844.0,2019,0.616154,0.175631,0.006250,0.023927,0.000891,0.120442,0.056706,0.252589
25,"Providence city, Rhode Island",179494.0,2019,0.550637,0.168318,0.009900,0.060097,0.001449,0.162891,0.046709,0.433201
34,"Woonsocket city, Rhode Island",41603.0,2019,0.735500,0.095858,0.009879,0.058073,0.000000,0.047064,0.053626,0.187919


### Poverty Metric

In [23]:
def poverty_metric(df):
  df['total_poverty_children'] = df['total_poverty_ratio_under1_children'] + df['total_poverty_ratio_1_to_2_children'] + df['total_poverty_ratio_2_or_more_children']
  df['total_poverty_families'] = df['poverty_ratio_under.5'] + df['poverty_ratio_.5_.74'] + df['poverty_ratio_.75_.99']
  df['total_poverty_1_1.99_families'] = df['poverty_ratio_1_1.24'] + df['poverty_ratio_1.25_1.49'] + df['poverty_ratio_1.5_1.74'] \
  + df['poverty_ratio_1.75_1.84'] + df['poverty_ratio_1.85_1.99']
  
  # proportion people below poverty level
  df['poverty_ratio_below_1_people']= df['all_in_poverty'] / df['total_poverty']

  # proportion familiies below poverty level method 2
  df['poverty_ratio_below_1_families']= df['total_poverty_families'] / df['ratio_income_to_poverty_families']
  df['poverty_ratio_below_2_families'] = (df['total_poverty_families'] + df['total_poverty_1_1.99_families'] )/ df['ratio_income_to_poverty_families']

  # proportion children below poverty level
  df['poverty_ratio_below_1_children'] = df['total_poverty_ratio_under1_children'] /df['ratio_of_income_to_poverty']
  df['poverty_ratio_below_2_children'] = (df['total_poverty_ratio_under1_children'] + df['total_poverty_ratio_1_to_2_children'])/df['total_poverty_children']
  
  # drop columns
  columns_to_drop = ['ratio_of_income_to_poverty',
                    'total_poverty_ratio_under1_children',
                    'total_poverty_ratio_1_to_2_children',
                    'total_poverty_ratio_2_or_more_children',
                    'ratio_income_to_poverty_families',
                    'poverty_ratio_under.5',
                    'poverty_ratio_.5_.74',
                    'poverty_ratio_.75_.99',
                    'poverty_ratio_1_1.24',
                    'poverty_ratio_1.25_1.49',
                    'poverty_ratio_1.5_1.74',
                    'poverty_ratio_1.75_1.84',
                    'poverty_ratio_1.85_1.99',
                    'total_poverty_1_1.99_families',
                    'total_poverty',
                    #'all_in_poverty', # don't drop this because need it for the number of people living in poverty in a given tract
                    'all_not_poverty',
                    'total_poverty_children',
                    'total_poverty_families',
                     ]
  df.drop(axis=1, columns=columns_to_drop,inplace=True)

  return df



all_df_place = poverty_metric(all_df_place)

us_tract_df = poverty_metric(us_tract_df)

df_county = poverty_metric(df_county)


### Number of Public School Students for Homeless Metric

In [24]:
def public_school_students(df):

  grades = ['preschool','kinder','1_4', '5_8','9_12']
  genders=['male','female']
  student_population = []

  for grade in grades:
    for gender in genders:
      student_population.append(gender + '_public_' + grade)
  
  df['public_students_pre_12'] = df[student_population].sum(axis=1)

  df.drop(student_population, axis=1,inplace=True)

  return df




all_df_place = public_school_students(all_df_place)

df_county = public_school_students(df_county)

### Employment Metric

In [25]:
def employment_metric(df):
  ages_early_late = ['16_19', '20_21', '22_24', '55_59', '60_61','62_64']
  ages = ['25_29', '30_34', '35_39', '40_44', '45_49', '50_54']
  ages_labor_force = ['25_29', '30_34', '35_44', '45_54']
  genders = ['male_', 'female_']
  
  races = ['white', 'black', 'american_indian', 'asian', 'some_other_race_alone', 'two_or_more_race', 'hispanic_or_latino']
  population_25_54 = []
  employed_25_54 = []
  population_16_24_55_64 = []
  employed_16_24_55_64 = []

  population_early_late = []
  employed_early_late = []
  population_16_64_race = []
  employed_16_64_race = []


  # create list of all columns of people age 25 - 54
  for gender in genders:
    for age in ages:
      population_25_54.append(gender+age)
  
  # create list of all columns of people age 16 - 24 and 55-64
  for gender in genders:
    for age in ages_early_late:
      population_early_late.append(gender+age)
      employed_early_late.append(gender+age+'_labor_force_employed')

  for gender in genders:
    for age in ages_labor_force:
      employed_25_54.append(gender+age+'_labor_force_employed')

  for race in races:
    df[race+'_employed_16_64'] = (df[race + '_male_16_64_employed'] + df[race + '_female_16_64_employed'])/(df[race + '_male_16_64'] + df[race + '_female_16_64'])
    for gender in genders: 
      population_16_64_race.append(race +'_' + gender+'16_64')
      employed_16_64_race.append(race+'_' + gender+'16_64_employed')
           

  # add those columns
  df['population_early_late'] = df[population_early_late].sum(axis=1)
  df['employed_early_late'] = df[employed_early_late].sum(axis=1)
  df['population_25_54'] = df[population_25_54].sum(axis=1)
  df['employed_25_54'] = df[employed_25_54].sum(axis=1)

  # find percent of all 25 to 54 year olds who are employed and find percent of the labor force who are employed
  df['employed_25_54_population'] = df['employed_25_54'] / df['population_25_54']
  df['employed_16_64_population'] = (df['employed_25_54'] + df['employed_early_late'])/ (df['population_25_54'] + df['population_early_late'])
  
  #drop columns
  df.drop(axis=1, columns =population_25_54, inplace=True)
  df.drop(axis=1, columns =employed_25_54, inplace=True)
  df.drop(axis=1, columns =population_early_late, inplace=True)
  df.drop(axis=1, columns =employed_early_late, inplace=True)
  df.drop(axis=1, columns =population_16_64_race, inplace=True)
  df.drop(axis=1, columns =employed_16_64_race, inplace=True)
  df.drop(axis=1, columns = ['population_early_late','employed_early_late', 'population_25_54', 'employed_25_54' ], inplace=True)

  return df


all_df_place = employment_metric(all_df_place)

df_county = employment_metric(df_county)
all_df_place.T

,3,24,25,34
name,"Central Falls city, Rhode Island","Pawtucket city, Rhode Island","Providence city, Rhode Island","Woonsocket city, Rhode Island"
population,19429.0,71844.0,179494.0,41603.0
white,0.485048,0.616154,0.550637,0.7355
black,0.126615,0.175631,0.168318,0.095858
native_american,0.010088,0.00625,0.0099,0.009879
...,...,...,...,...
some_other_race_alone_employed_16_64,0.651511,0.699946,0.662459,0.541292
two_or_more_race_employed_16_64,0.541763,0.675836,0.578303,0.664234
hispanic_or_latino_employed_16_64,0.639005,0.682438,0.635338,0.551724
employed_25_54_population,0.693516,0.784585,0.731694,0.723075


## Preschool Metric

In [26]:
def preschool_clean(df):
  #note: % enrolled in preschool = 1-(all 3&4 year olds not enrolled)/(all 3 & 4 year olds)
  # preschool_enroll is percent of students enrolled in preschool
  df['preschool_enroll']=1-(df['mal_not_enrolled_3_4']+df['fem_not_enrolled_3_4'])/df['total_3_4']

  # this is the number of students enrolled by each race
  preschool_race = ['preschool_enrollment_white', 'preschool_enrollment_black',
        'preschool_enrollment_hispanic', 'preschool_enrollment_indigenous',
        'preschool_enrollment_asian']

  # find percent of preschool students of each racial/ethnic category
  for race in preschool_race:
      df[race+'_3_4'] = df[race]/df['total_3_4']
      df[race]= df[race]/df['preschool_enrollment']
      

  races = ['white','black','indigenous','asian','hispanic', 'two_or_more_race', 'some_other_race']

  
  # find percent of under 5s of each racial/ethnic category
  for race in races:
      df[race+'_under_5']=(df[race+'_m_under5']+df[race+'_f_under5'])/(df['female_under5'] + df['male_under5'])
      
      #find dif. in enrollment: % in preschool - percent under 5
      # df['dif_enrol_'+race] = df['preschool_enrollment_'+race]-df[race+'_under_5']

  # find approximation for each race of percent enrolled in preschool
  df['percent_white_enrolled_preschool'] = df['preschool_enrollment_white_3_4']/df['white_under_5']
  df['percent_black_enrolled_preschool'] = df['preschool_enrollment_black_3_4']/df['black_under_5']
  df['percent_asian_enrolled_preschool'] = df['preschool_enrollment_asian_3_4']/df['asian_under_5']
  df['percent_hispanic_enrolled_preschool'] = df['preschool_enrollment_hispanic_3_4']/df['hispanic_under_5']


  to_drop_preschool = ['preschool_enrollment', 'female_under5', 'male_under5','white_m_under5', 'white_f_under5','black_m_under5', 'black_f_under5','indigenous_m_under5',\
                       'indigenous_f_under5','asian_m_under5','asian_f_under5','hispanic_m_under5', 'hispanic_f_under5', 'two_or_more_race_m_under5', 'two_or_more_race_f_under5','total_3_4',\
                       "fem_not_enrolled_3_4","mal_not_enrolled_3_4", 'preschool_enrollment_white_3_4', 'preschool_enrollment_black_3_4', 'preschool_enrollment_hispanic_3_4',\
                       'preschool_enrollment_indigenous_3_4', 'preschool_enrollment_asian_3_4', 'hawaiian_m_under5', 'hawaiian_f_under5', 'some_other_race_m_under5', 'some_other_race_f_under5'
                       ]
  drop redundent columns
  df.drop(axis = 1, columns=to_drop_preschool, inplace=True)

  return df



all_df_place = preschool_clean(all_df_place)

df_county = preschool_clean(df_county)



In [44]:

preschool_metrics = ['name', 'year', 'preschool_enroll','preschool_enrollment_white', 'percent_white_enrolled_preschool','preschool_enrollment_black','percent_black_enrolled_preschool',
        'preschool_enrollment_hispanic', 'percent_hispanic_enrolled_preschool', 'preschool_enrollment_indigenous',
        'preschool_enrollment_asian','percent_asian_enrolled_preschool', 'white_under_5','black_under_5','indigenous_under_5','asian_under_5','hispanic_under_5'
        ]

#preschool_metrics = ['name', 'state','county', 'year', 'preschool_enroll','preschool_enrollment','total_3_4', 'mal_not_enrolled_3_4', 'fem_not_enrolled_3_4']


df_county[preschool_metrics].sort_values(by='preschool_enroll')['name'].iloc[0]


#df_county[df_county['name']=='Grant County, Nebraska']['preschool_enroll'] = 0
# df_county[df_county['name']=='Grant County, Nebraska']['preschool_enroll']

# df_county[df_county['name']=='Grant County, Nebraska']

df_county.loc[1691,'preschool_enroll']=0


0.0

In [45]:
# import altair as alt

# under5 = alt.Chart(all_df_place).mark_line().encode(
#     x='year',
#     y='hispanic_under_5',
#     color='name',
#     tooltip='name'
# )

# latino = alt.Chart(all_df_place).mark_line().encode(
#     x='year:O',
#     y='dif_enrol_hispanic',
#     color='name',
#     tooltip='name'
# )

# black = alt.Chart(all_df_place).mark_line().encode(
#     x='year:Q',
#     y='dif_enrol_black',
#     color='name',
#     tooltip='name'
# )
# latino

In [46]:
# def preschool_chart(df,location,race,year):
#     #df_location = df[df['name']==location +' city, Rhode Island']
#     df_year = df[df['year']==year]
#     df_melt = pd.melt(df_year, id_vars=['name'], value_vars=['preschool_enrollment_'+ race,race +'_under_5'])

#     chart = alt.Chart(df_melt).mark_bar().encode(
#     column=alt.Column('name'),
#     x=alt.X('variable',title=None),
#     y=alt.Y('value',axis=alt.Axis(format='%',grid=False),title='Percent '+race),
#     color=alt.Color('variable', scale=alt.Scale(range=['#EA98D2', '#659CCA']),legend=alt.Legend(title='Percent '+ race)),
#     tooltip='value'
#     ).configure_view(
#         strokeWidth=0.0,
#     ).properties(title=location + ' ' + race + ' Preschool Enrollment Increases Over Time')

#     return chart

# preschool_chart(all_df_place,'Pawtucket','black',2018)

In [47]:
columns_to_drop = ['GEO_ID', 'median_family_income_white_not_hisp', 'poverty_ratio_below_1_people', 'poverty_ratio_below_1_families',
'poverty_ratio_below_2_families', 'poverty_ratio_below_1_children', 'poverty_ratio_below_2_children', 'percent_white_enrolled_preschool',
'percent_black_enrolled_preschool', 'percent_asian_enrolled_preschool', 'percent_hispanic_enrolled_preschool', 'under_18']
df_county.drop(columns = columns_to_drop, inplace=True)
for column in list(df_county.columns):
  print(column)

name
population
white
black
native_american
asian
hawaiian
some_other_race_alone
two_more_races
hispanic_or_latino
state
county
median_family_income
income_20_percentile
income_80_percentile
median_family_income_white
median_family_income_black
median_family_income_indigenous
median_family_income_asian
median_family_income_hispanic
female_under5
male_under5
white_m_under5
white_f_under5
black_m_under5
black_f_under5
indigenous_m_under5
indigenous_f_under5
asian_m_under5
asian_f_under5
hawaiian_m_under5
hawaiian_f_under5
some_other_race_m_under5
some_other_race_f_under5
two_or_more_race_m_under5
two_or_more_race_f_under5
hispanic_m_under5
hispanic_f_under5
total_3_4
preschool_enrollment
preschool_enrollment_white
preschool_enrollment_black
preschool_enrollment_hispanic
preschool_enrollment_indigenous
preschool_enrollment_asian
fem_not_enrolled_3_4
mal_not_enrolled_3_4
all_in_poverty
year
public_students_pre_12
white_employed_16_64
black_employed_16_64
american_indian_employed_16_64
asia

### Create CSV Files

In [48]:


all_df_place.to_csv(f'{data_path}/interim/ri_place_acs.csv', index=False)
us_tract_df.to_csv(f'{data_path}/interim/ri_tract_acs.csv', index=False)
df_county.to_csv(f'{data_path}/interim/us_county_acs.csv', index=False)
voting_population.to_csv(f'{data_path}/interim/voting_population_2020_acs.csv', index=False)
voting_population_ri_places.to_csv(f'{data_path}/interim/voting_population_ri_places_2020_acs.csv', index=False)

In [49]:
all_df_place.head()

,name,population,white,black,native_american,asian,hawaiian,some_other_race_alone,two_more_races,hispanic_or_latino,...,black_under_5,indigenous_under_5,asian_under_5,hispanic_under_5,two_or_more_race_under_5,some_other_race_under_5,percent_white_enrolled_preschool,percent_black_enrolled_preschool,percent_asian_enrolled_preschool,percent_hispanic_enrolled_preschool
3,"Central Falls city, Rhode Island",19429.0,0.485048,0.126615,0.010088,0.006485,0.004581,0.296413,0.070770,0.664110,...,0.127463,0.000000,0.003695,0.791872,0.106527,0.302956,0.272993,0.277490,2.610932,0.450721
24,"Pawtucket city, Rhode Island",71844.0,0.616154,0.175631,0.006250,0.023927,0.000891,0.120442,0.056706,0.252589,...,0.228070,0.005614,0.015906,0.372398,0.117661,0.143158,0.550459,0.254728,0.000000,0.514237
25,"Providence city, Rhode Island",179494.0,0.550637,0.168318,0.009900,0.060097,0.001449,0.162891,0.046709,0.433201,...,0.148431,0.009140,0.036734,0.612883,0.112136,0.199578,0.584674,0.573742,0.667390,0.415452
34,"Woonsocket city, Rhode Island",41603.0,0.735500,0.095858,0.009879,0.058073,0.000000,0.047064,0.053626,0.187919,...,0.144957,0.011624,0.029060,0.335726,0.108376,0.099829,0.483568,0.656246,0.878258,0.444604


In [50]:
df_county.T

#pop_white_pct
#drop pop race
# drop GEO_ID
# all_in_poverty

# (# kids white and in preschool / # 3 &4 year olds)/(% of kids under 5 who are white)


,0,1,2,3,4,5,6,7,8,9,...,3210,3211,3212,3213,3214,3215,3216,3217,3218,3219
name,"Autauga County, Alabama","Baldwin County, Alabama","Barbour County, Alabama","Bibb County, Alabama","Blount County, Alabama","Bullock County, Alabama","Butler County, Alabama","Calhoun County, Alabama","Chambers County, Alabama","Cherokee County, Alabama",...,"Toa Alta Municipio, Puerto Rico","Toa Baja Municipio, Puerto Rico","Trujillo Alto Municipio, Puerto Rico","Utuado Municipio, Puerto Rico","Vega Alta Municipio, Puerto Rico","Vega Baja Municipio, Puerto Rico","Vieques Municipio, Puerto Rico","Villalba Municipio, Puerto Rico","Yabucoa Municipio, Puerto Rico","Yauco Municipio, Puerto Rico"
population,55380.0,212830.0,25361.0,22493.0,57681.0,10248.0,19828.0,114618.0,33660.0,25903.0,...,72864.0,77810.0,66338.0,28676.0,37106.0,52192.0,8642.0,22403.0,33499.0,35428.0
white,0.767913,0.862054,0.468002,0.767883,0.954595,0.21897,0.518459,0.736699,0.577035,0.928541,...,0.826718,0.724303,0.794311,0.528456,0.71444,0.832062,0.547327,0.567469,0.186394,0.751016
black,0.190285,0.092647,0.47577,0.222914,0.016088,0.748048,0.453954,0.207908,0.395603,0.045979,...,0.072121,0.113983,0.100546,0.010322,0.105805,0.049605,0.07533,0.034504,0.788621,0.037993
native_american,0.002528,0.007729,0.003233,0.001334,0.000797,0.0,0.000555,0.003307,0.003327,0.01081,...,0.001441,0.00027,0.000829,0.000663,0.0,0.001226,0.000926,0.0,0.0,0.004488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
indigenous_under_5,0.011501,0.0,0.0,0.0,0.001166,0.0,0.0,0.004691,0.0,0.05813,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012908
asian_under_5,0.0,0.009582,0.007413,0.0,0.0,0.042435,0.005222,0.006356,0.0,0.0,...,0.0,0.0,0.007432,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hispanic_under_5,0.048803,0.092822,0.099333,0.081369,0.1588,0.077491,0.022628,0.070521,0.117964,0.026959,...,0.993432,0.996478,0.995169,0.979757,1.0,0.960085,0.901149,1.0,0.987925,1.0
two_or_more_race_under_5,0.026733,0.021131,0.040771,0.0,0.025058,0.086716,0.100087,0.064165,0.006363,0.01011,...,0.047291,0.067215,0.021925,0.051822,0.022485,0.008493,0.0,0.31446,0.0,0.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=96aa4e71-9142-4f1e-8c2a-54e13d3b1860' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>